In [29]:
import pandas as pd
from helper import pickle_store, pickle_restore, get_anom_list 
from adem import *


count = 0
#f4 to f6 dev, f5 to f9 logon
feature_name_dic = {'f1':'email_freq',  'f2':'outcmnty_ef',  'f3':'incmnty_ef',  'f4':'num_usage',  'f5':'num_pc', 'f6':'num_usage_posthr',  'f7':'num_usage',  'f8':'num_pc',  'f9':'num_usage_posthr',  'f10':'num_filecopy', 'f11':'num_pc'  }
feature_name = 'num_usage'
wins= 10
sigm=7

def pbp_getanom_ev(ubp_feature_dic, pbp_feature_dic, feature_name, ft):
    # print('PBP')
    pbp_events_dic={}
    for key in ubp_feature_dic:
        usr=key #'AAM0658'
        debug_df = ubp_feature_dic[usr]
        debug_peer_df = pbp_feature_dic[usr_peer_map[usr]]

        #do a depp copy of preped feature dataframe
        df = debug_df.copy()
        df_peer = debug_peer_df.copy()
        
        uname = key
        # print(key)
        anom_ev = anom_calc_pbp(df, df_peer, feature_name, uname, ws=wins, sig=sigm, fti = ft)

        if len(anom_ev['anomalies_dict']):
            pbp_events_dic[key]=anom_ev

    return pbp_events_dic

def cbp_getanom_ev(ubp_feature_dic, cbp_feature_dic, feature_name, ft):
    # print('CBP')
    cbp_events_dic={}
    for key in ubp_feature_dic:
        usr=key
        debug_df = ubp_feature_dic[usr]
        debug_cmnty_df = cbp_feature_dic[usr_cmnty_map[usr]]

        #do a depp copy of preped feature dataframe
        df = debug_df.copy()
        df_cmnty = debug_cmnty_df.copy()

        uname = key
        # print(key)
        anom_ev = anom_calc_cbp(df, df_cmnty, feature_name, uname, ws=wins, sig=sigm, fti= ft)

        if len(anom_ev['anomalies_dict']):
            cbp_events_dic[key]=anom_ev

    return cbp_events_dic
        


def ubp_getanom_ev(ubp_feature_dic, feature_name, ft):
    # print('UBP')
    ubp_events_dic={}
    # email_freq_feature_dic.keys()
    # Feature data format:  user: df [user, date, featurecolmn]
    for key in ubp_feature_dic:
        usr=key
        debug_df = ubp_feature_dic[usr]


        #do a depp copy of preped feature dataframe
        df = debug_df.copy()
        # print(key)
        anom_ev = anom_calc_ubp(df, feature_name, usr, ws=wins, sig=sigm, fti= ft)

        if len(anom_ev['anomalies_dict']):
            ubp_events_dic[key]=anom_ev

    return ubp_events_dic


In [30]:
# fetch ubp, pbp and cbp dics of each feature and get anom_events and save the anom list in f_i 

import os

usr_cmnty_map = pickle_restore("pickle/community_louvian_file")
usr_peer_map = pickle_restore("pickle/eid_role_map_file")



def get_anom_user_list(ubp_events_dic, pbp_events_dic, cbp_events_dic):
    anom_user_dic={}

    anom_user_dic['ubp'] = get_anom_list(ubp_events_dic)
    anom_user_dic['pbp'] = get_anom_list(pbp_events_dic)
    anom_user_dic['cbp'] = get_anom_list(cbp_events_dic)

    return anom_user_dic


feature_path = 'pickle/'

fi_path=[]
for subdir, dirs, files in os.walk(feature_path):
    # print ("subdir=",subdir)
    # print ("dir=",dirs)
    for file in dirs:
        fi_path.append(file)
    break

print("fi_path=", fi_path)

#remove because they cause huge number of false postives f2 900, f3 700
fi_path.remove('f2')
fi_path.remove('f3')

for fi in fi_path: #['f4']
    print('*********************************************')
    print(fi)
    print('*********************************************')

    for subfolder, folder, files in os.walk('pickle/'+fi):
        # print (files)
        fi_ubp_dic = pickle_restore('pickle/'+fi+'/'+fi+'_ubp_file')# fi_ubp_file
        fi_pbp_dic = pickle_restore('pickle/'+fi+'/'+fi+'_pbp_file')
        fi_cbp_dic = pickle_restore('pickle/'+fi+'/'+fi+'_cbp_file')
        # fi_anom_set_dic = pickle_restore('pickle/'+fi+'/'+fi+'_anom_set_file')


        ubp_events_dic = ubp_getanom_ev(fi_ubp_dic, feature_name_dic[fi], fi)
        pbp_events_dic = pbp_getanom_ev(fi_ubp_dic, fi_pbp_dic, feature_name_dic[fi], fi)
        cbp_events_dic = cbp_getanom_ev(fi_ubp_dic, fi_cbp_dic, feature_name_dic[fi], fi)


        anom_user_lst_dic = get_anom_user_list(ubp_events_dic, pbp_events_dic, cbp_events_dic)

        pickle_store('pickle/'+fi+'/'+fi+'_anom_set_file', anom_user_lst_dic) #raw means not indexed by Timeindex



fi_path= ['f6', 'f1', 'f5', 'f2', 'f8', 'f11', 'f7', 'f10', 'f9', 'f3', 'f4']
*********************************************
f6
*********************************************
*********************************************
f1
*********************************************
*********************************************
f5
*********************************************
*********************************************
f8
*********************************************
*********************************************
f11
*********************************************
*********************************************
f7
*********************************************
*********************************************
f10
*********************************************
*********************************************
f9
*********************************************
*********************************************
f4
*********************************************
